In [5]:
# Import des librairies
import requests
import pandas as pd
import numpy as np
import os
from typing import List
from steps.ingest import get_full_text, get_new_images_to_ocerize, get_processed_dataset, save_text_to_file
ocr_endpoint = "http://localhost:8901/txt/blocks-words" # url de l'OCR
data_path = "../data/" # chemin du dossier data

In [9]:
# Appel à l'OCR (TEST)
test_image_path = "../data/raw/final/facture_test.png" 
result = get_full_text(test_image_path)
print(result)

Facture
Logo
Vendeur
Mon Entreprise 22, Avenue Voltaire 13000 Marseille Michel Acheteur 31, rue de la Forêt 13100 Alx-en-Provencs
Client
Date de facturation Numéro de facture Échéance
Paiement 30 jours
Référence 1436
2.6.2021
143
16.6.2021
Informations additionnalles : Service Après Vente : Garantie 1 an.
Description Main-d'oeuvre Produit
Quantité 5 10
Unité h pcs
Prix unitaire HT % TVA
Total TVA Total TTC
00,00  105,00 €
20 % 20 %
00,00 € 210,00 € Total HT Total TVA Total TTC
300,00 € 1 260,00 € 1 350,00 € 270,00 € 1 620,00 €
Mon Entreprise 22, Avenue Voitare 13000 Marsellle N" Siren ou Siret 24567 N TVA intra - FRXX 999999999
onordtinnnéas Pieme Fomisees Telephone 2 4 6 & & 99 E-mall Pemegmacompagnien WwWW macompagnie. com
petalle bancairen anbueg BAN BIC/SwI
NP Paribas FR23 4112 4098 4098 23 00HCs


In [21]:
# Appel au clean
clean_url = "http://localhost:8903/clean" # url du cleaner/tokenizer
def get_cleaned_text(text) -> str:
    print(clean_url)
    headers = {'Content-Type': 'text/plain'}
    params = {
        "text": text
    }
    response = requests.post(clean_url, params=params, headers=headers)
    return response
get_cleaned_text(result).text

http://localhost:8903/clean


"factur logo vendeur entreprise 22 avenue voltair 13000 marseill michel acheteur 31 rue forêt 13100 alx-en-provenc client date facturation numéro facture échéance paiemer 30 jour référence 1436 2.6.2021 143 16.6.2021 information additionnalle service vente garantie an description main - oeuvre produire quantité 10 unité pc prix unitaire ht tva total ttc 00,00 105,00 20 210,00 300,00 260,00 350,00 270,00 620,00 voitare marselll '' siren siret 24567 intrer frxx 999999999 onordtinnnéer pieme fomiseer telephone 99 e-mall pemegmacompagnien wwww macompagnie com petalle bancairen anbueg ban bic sur swi np pariba fr23 4112 4098 23 00hcs"

In [22]:
# TEST INGEST EN MASSE
path_to_dataset = f"{data_path}/processed/processed_dataset.csv"
raw_dataset = pd.read_csv(f"{data_path}dataset.csv")
processed_dataset = get_processed_dataset(path_to_dataset)

new_images = get_new_images_to_ocerize(raw_dataset, processed_dataset)

for _, row in new_images.head(10).iterrows():                                               # TODO : Attention j'ai mis head(10) pour tester, à enlever
    full_text = get_full_text(f"{data_path}raw/final/{row.filename}")

    text_file_path = f"{data_path}/processed/{row.filename}.txt"
    save_text_to_file(full_text, text_file_path)
    new_images.loc[new_images['filename']==row.filename, 'full_text'] = text_file_path 

processed_dataset = pd.concat([processed_dataset, new_images.head(10)],ignore_index=True)   # TODO : Attention j'ai mis head(10) pour tester, à enlever
processed_dataset.to_csv(path_to_dataset, index=None)

/tmp/ipykernel_34710/3157561810.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_images.loc[new_images['filename']==row.filename, 'full_text'] = text_file_path
